In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

dataset = pd.read_csv("credito4.csv", sep = ";")
dataset.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,CLASSE
0,1169.0,4,67,nenhum,01/01/2019,masculino solteiro,radio/tv,bom
1,5951.0,2,22,nenhum,01/01/2020,fem div/cas,radio/tv,ruim
2,2096.0,3,49,nenhum,02/01/2020,masculino solteiro,educação,bom
3,7882.0,4,45,nenhum,02/01/2019,masculino solteiro,mobilia/equipamento,bom
4,4870.0,4,53,nenhum,03/01/2018,masculino solteiro,carro novo,ruim


In [13]:
X = dataset.iloc[:, 0:7]
y = dataset["CLASSE"]

In [12]:
# Tratamento de valores faltantes
X.isnull().sum()

SALDO_ATUAL         7
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         8
PROPOSITO           0
dtype: int64

In [15]:
# O saldo será preenchido pela mediana
mediana = X["SALDO_ATUAL"].median()
mediana

2323.0

In [16]:
X["SALDO_ATUAL"].fillna(mediana, inplace = True)
X.isnull().sum()

SALDO_ATUAL         0
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         8
PROPOSITO           0
dtype: int64

In [17]:
# Verifica qual a moda do estado civil
agrupado = X.groupby(["ESTADOCIVIL"]).size()
agrupado

ESTADOCIVIL
fem div/cas               308
masculino casado/viuvo     92
masculino div/sep          50
masculino solteiro        542
dtype: int64

In [18]:
# Substitui o estado civil pela moda
X["ESTADOCIVIL"].fillna("masculino solteiro", inplace = True)
X.isnull().sum()

SALDO_ATUAL         0
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         0
PROPOSITO           0
dtype: int64

In [19]:
# Outliers, definimos como acima de 2 std
desv = X["SALDO_ATUAL"].std()
desv

685936688.9820064

In [20]:
# Checamos se algum SALDO_ATUAL atende ao critério
X.loc[X["SALDO_ATUAL"] >= 2 * desv, "SALDO_ATUAL"]

127    2.541111e+09
160    2.154441e+10
Name: SALDO_ATUAL, dtype: float64

In [22]:
# Vamos atualizar o saldo para a mediana
mediana = X["SALDO_ATUAL"].median()
mediana

2323.0

In [23]:
# Atribuimos
X.loc[X["SALDO_ATUAL"] >= 2 * desv, "SALDO_ATUAL"] = mediana
X.loc[X["SALDO_ATUAL"] >= 2 * desv]

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO


In [27]:
# Binning
agrupado = X.groupby(["PROPOSITO"]).size()
agrupado

PROPOSITO
Eletrodomésticos        12
carro novo             234
carro usado            103
educação                50
mobilia/equipamento    181
negócios                97
obras                   22
outros                  12
qualificação             9
radio/tv               280
dtype: int64

In [28]:
X.loc[X["PROPOSITO"] == "Eletrodomésticos", "PROPOSITO"] = "outros"
X.loc[X["PROPOSITO"] == "mobilia/equipamento", "PROPOSITO"] = "outros"
X.loc[X["PROPOSITO"] == "educação", "PROPOSITO"] = "outros"
agrupado = X.groupby(["PROPOSITO"]).size()
agrupado

PROPOSITO
carro novo      234
carro usado     103
negócios         97
obras            22
outros          255
qualificação      9
radio/tv        280
dtype: int64

In [29]:
# Extração de característica da data
X["DATA"] = pd.to_datetime(X["DATA"], format = "%d/%m/%Y")
X["DATA"]

0     2019-01-01
1     2020-01-01
2     2020-01-02
3     2019-01-02
4     2018-01-03
         ...    
995   2018-06-29
996   2018-06-30
997   2018-07-03
998   2019-07-04
999   2018-07-05
Name: DATA, Length: 1000, dtype: datetime64[ns]

In [30]:
# 3 Novos atributos
X["ANO"] = X["DATA"].dt.year
X["MES"] = X["DATA"].dt.month
X["DIASEMANA"] = X["DATA"].dt.day_name()

X["DIASEMANA"]

0        Tuesday
1      Wednesday
2       Thursday
3      Wednesday
4      Wednesday
         ...    
995       Friday
996     Saturday
997      Tuesday
998     Thursday
999     Thursday
Name: DIASEMANA, Length: 1000, dtype: object

In [31]:
# Label encoder em estado civil, proposito e dia da semana
X["ESTADOCIVIL"].unique()

array(['masculino solteiro', 'fem div/cas', 'masculino div/sep',
       'masculino casado/viuvo'], dtype=object)

In [32]:
X["DIASEMANA"].unique()

array(['Tuesday', 'Wednesday', 'Thursday', 'Saturday', 'Sunday', 'Monday',
       'Friday'], dtype=object)

In [33]:
X["PROPOSITO"].unique()

array(['radio/tv', 'outros', 'carro novo', 'carro usado', 'negócios',
       'obras', 'qualificação'], dtype=object)

In [35]:
labelencoder = LabelEncoder()
X["ESTADOCIVIL"] = labelencoder.fit_transform(X["ESTADOCIVIL"])
X["DIASEMANA"] = labelencoder.fit_transform(X["DIASEMANA"])
X["PROPOSITO"] = labelencoder.fit_transform(X["PROPOSITO"])

X.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA
0,1169.0,4,67,nenhum,2019-01-01,3,6,2019,1,5
1,5951.0,2,22,nenhum,2020-01-01,0,6,2020,1,6
2,2096.0,3,49,nenhum,2020-01-02,3,4,2020,1,4
3,7882.0,4,45,nenhum,2019-01-02,3,4,2019,1,6
4,4870.0,4,53,nenhum,2018-01-03,3,0,2018,1,6


In [37]:
# One hot encoding em Outros planos de pagamento
outros = X["OUTROSPLANOSPGTO"].unique()
outros

array(['nenhum', 'banco', 'stores'], dtype=object)

In [39]:
z = pd.get_dummies(X["OUTROSPLANOSPGTO"], prefix = "OUTROS")
z

,OUTROS_banco,OUTROS_nenhum,OUTROS_stores
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
...,...,...,...
995,0,1,0
996,0,1,0
997,0,1,0
998,0,1,0


In [40]:
# Padronização Z-score
sc = StandardScaler()
m = sc.fit_transform(X.iloc[:, 0:3])

m

array([[-0.74551643,  1.04698668,  1.6392759 ],
       [ 0.95774038, -0.76597727, -0.74024139],
       [-0.41533679,  0.14050471,  0.68746898],
       ...,
       [-0.87552244,  1.04698668,  0.1058092 ],
       [-0.50473818,  1.04698668, -0.68736323],
       [ 0.46799171,  1.04698668, -0.47585058]])

In [41]:
# Concatenação dos dados
X = pd.concat([X, z, pd.DataFrame(m, columns = ["SALDO_ATUAL_N", "RESIDENCIADESDE_N", "IDADE_N"])], axis = 1)

X

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA,OUTROS_banco,OUTROS_nenhum,OUTROS_stores,SALDO_ATUAL_N,RESIDENCIADESDE_N,IDADE_N
0,1169.0,4,67,nenhum,2019-01-01,3,6,2019,1,5,0,1,0,-0.745516,1.046987,1.639276
1,5951.0,2,22,nenhum,2020-01-01,0,6,2020,1,6,0,1,0,0.957740,-0.765977,-0.740241
2,2096.0,3,49,nenhum,2020-01-02,3,4,2020,1,4,0,1,0,-0.415337,0.140505,0.687469
3,7882.0,4,45,nenhum,2019-01-02,3,4,2019,1,6,0,1,0,1.645526,1.046987,0.475956
4,4870.0,4,53,nenhum,2018-01-03,3,0,2018,1,6,0,1,0,0.572709,1.046987,0.898982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1736.0,4,31,nenhum,2018-06-29,0,4,2018,6,0,0,1,0,-0.543562,1.046987,-0.264338
996,3857.0,4,40,nenhum,2018-06-30,2,1,2018,6,2,0,1,0,0.211898,1.046987,0.211566
997,804.0,4,38,nenhum,2018-07-03,3,6,2018,7,5,0,1,0,-0.875522,1.046987,0.105809
998,1845.0,4,23,nenhum,2019-07-04,3,6,2019,7,4,0,1,0,-0.504738,1.046987,-0.687363


In [42]:
# Remoção de algumas colunas
X.drop(columns = ["SALDO_ATUAL", "RESIDENCIADESDE", "IDADE", "OUTROSPLANOSPGTO", "DATA", "OUTROS_banco"], inplace = True)

X

,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA,OUTROS_nenhum,OUTROS_stores,SALDO_ATUAL_N,RESIDENCIADESDE_N,IDADE_N
0,3,6,2019,1,5,1,0,-0.745516,1.046987,1.639276
1,0,6,2020,1,6,1,0,0.957740,-0.765977,-0.740241
2,3,4,2020,1,4,1,0,-0.415337,0.140505,0.687469
3,3,4,2019,1,6,1,0,1.645526,1.046987,0.475956
4,3,0,2018,1,6,1,0,0.572709,1.046987,0.898982
...,...,...,...,...,...,...,...,...,...,...
995,0,4,2018,6,0,1,0,-0.543562,1.046987,-0.264338
996,2,1,2018,6,2,1,0,0.211898,1.046987,0.211566
997,3,6,2018,7,5,1,0,-0.875522,1.046987,0.105809
998,3,6,2019,7,4,1,0,-0.504738,1.046987,-0.687363


In [43]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size = 0.3, random_state = 0)
floresta = RandomForestClassifier(n_estimators = 100)
floresta.fit(X_treino, y_treino)
previsoes = floresta.predict(X_teste)
confusao = confusion_matrix(y_teste, previsoes)
taxa_acerto = accuracy_score(y_teste, previsoes)

confusao

array([[189,  25],
       [ 63,  23]], dtype=int64)

In [44]:
taxa_acerto

0.7066666666666667